This is going to randomly sample, but then also return consective data points. both based on a dim

In [3]:
import torch
from torch.utils.data import Dataset, Sampler, DataLoader
import numpy as np

class WaveformDataset(Dataset):
    def __init__(self, wav_data, t_input):
        self.wav_data = wav_data
        self.time_steps = t_input
        self.length = len(wav_data)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        return self.wav_data[idx], self.time_steps[idx]


In [62]:
class RandomConsecutiveSampler(Sampler):
    def __init__(self, data_source, batch_size, consecutive_size):
        self.data_source = data_source
        self.batch_size = batch_size
        self.consecutive_size = consecutive_size

    def __iter__(self):
        indices = np.random.permutation(len(self.data_source) - self.consecutive_size + 1)
        for i in range(0, len(indices), self.batch_size):
            batch_indices = []
            for j in range(i, min(i + self.batch_size, len(indices))):
                start_idx = indices[j]
                batch_indices.extend(range(start_idx, start_idx + self.consecutive_size))
            yield batch_indices

    def __len__(self):
        return (len(self.data_source) - self.consecutive_size ) // self.batch_size


    
    

wav_data = np.linspace(1,100,100).tolist()  # Example waveform data
t_input = np.linspace(1,100,100).tolist()  # Example time step data

dataset = WaveformDataset(wav_data, t_input)
batch_size = 17
consecutive_size = 13

sampler = RandomConsecutiveSampler(dataset, batch_size, consecutive_size)
data_loader = DataLoader(dataset, batch_sampler=sampler)



In [63]:
def consecutive_dif(prediction, target, consecutive_size):
    """
    This will calculate the differences across consecutive points.  
    use RandomConsecutiveSampler for intended results
    """
    pred_reshape = prediction.view(-1,consecutive_size)
    target_reshape = target.view(-1,consecutive_size)
    pred_dif = pred_reshape[:,1:] - pred_reshape[:,:-1]
    target_dif = target_reshape[:,1:] - target_reshape[:,:-1]
    return torch.mean((pred_dif - target_dif)**2)
    
#variations.  you can do the differences with all the other consecutive points.  
# you could do differences along the batch.  those are consecutive points but they shift one each time
# weight those with the dif in t.  that could be interesting.  
#might add in another dataloader? a completely random one? what if you use mse on the random one? 
#thats interesting.  or maybe just weird.  
#no its interesting.  you learn the slopes from differences, then the vertical shift from mse on different points.  
# try these on a single wave to learn frequency.  then add in phi?


In [64]:
# Iterate through the data loader
for batch in data_loader:
    waveforms, time_steps = batch
    
    x = consecutive_dif(waveforms,waveforms+torch.rand_like(waveforms),consecutive_size)
    print(x)
    break

tensor(0.1520, dtype=torch.float64)


In [ ]:
#for tomorrow, i want to make the layer that duplicates the input t, 
#and applies different A and phi, then sums them to make one output t. 
#so your first layers have lots and lots, and subsequent layers have few.  
#that is going to be a tree.  i want to specify num branches and num layers.  

#later we try it with triangle waves
